In [18]:
## Getting the list of files
from pathlib import Path
fish_dir = Path('./fishes/Fish_Dataset/Fish_Dataset')
file_path = list(fish_dir.glob(r"**/*.png"))

file_path[:10]

[WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00001.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00002.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00003.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00004.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00005.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00006.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00007.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00008.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00009.png'),
 WindowsPath('fishes/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00010.png')]

In [19]:
## Getting the labels of the images
from os import path
import pandas as pd

labels = list(map(lambda f: path.split(path.split(f)[0])[1], file_path))

labels = pd.Series(labels)
labels.unique()

array(['Black Sea Sprat', 'Black Sea Sprat GT', 'Gilt-Head Bream',
       'Gilt-Head Bream GT', 'Hourse Mackerel', 'Hourse Mackerel GT',
       'Red Mullet', 'Red Mullet GT', 'Red Sea Bream', 'Red Sea Bream GT',
       'Sea Bass', 'Sea Bass GT', 'Shrimp', 'Shrimp GT',
       'Striped Red Mullet', 'Striped Red Mullet GT', 'Trout', 'Trout GT'],
      dtype=object)

In [20]:
## Images series
images = pd.Series(file_path).astype(str)
images.head()

0    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
1    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
2    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
3    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
4    fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...
dtype: object

In [21]:
## Creating dataframe of fishes with their labels
fishes = pd.concat([images, labels], axis=1)
fishes.columns = ["image", "label"]
fishes = fishes[fishes["label"].apply(lambda x: x[-2:] != "GT")].reset_index(drop=True)
fishes.shape

(9000, 2)

In [22]:
fishes.head()

,image,label
0,fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...,Black Sea Sprat
1,fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...,Black Sea Sprat
2,fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...,Black Sea Sprat
3,fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...,Black Sea Sprat
4,fishes\Fish_Dataset\Fish_Dataset\Black Sea Spr...,Black Sea Sprat


In [23]:
## Splitting data
from sklearn.model_selection import train_test_split

train_set, test_data = train_test_split(fishes, test_size=0.3, random_state=42)
train_data, val_data = train_test_split(train_set, test_size=0.2, random_state=42)

print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(5040, 2)
(2700, 2)
(1260, 2)


In [24]:
## Image Generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
input_shape = (224, 224, 3)

img_gen = ImageDataGenerator(rotation_range=15, rescale=1. / 255, horizontal_flip=True)

train_gen = img_gen.flow_from_dataframe(train_data,
                                        x_col="image",
                                        y_col="label",
                                        target_size=img_size,
                                        class_mode="categorical",
                                        batch_size=32)

validation_gen = img_gen.flow_from_dataframe(val_data,
                                             x_col="image",
                                             y_col="label",
                                             target_size=img_size,
                                             class_mode="categorical",
                                             batch_size=32,
                                             shuffle=False)

Found 5040 validated image filenames belonging to 9 classes.
Found 1260 validated image filenames belonging to 9 classes.


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D

model = Sequential([Conv2D(32, (3, 3), activation='relu', strides=(1, 1), input_shape=input_shape),
                    MaxPooling2D(pool_size=(2, 2)), Flatten(),
                    Dense(128, activation='relu'), Dropout(0.2),
                    Dense(128, activation='relu'), Dropout(0.2), Dense(9, activation="softmax")])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 394272)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               50466944  
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [26]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [27]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger

rate_reduction = ReduceLROnPlateau(monitor="val_accuracy", patience=5, verbose=1, factor=0.2, min_lr=0.0001)
early_stop = EarlyStopping(patience=10)
csv_logger = CSVLogger("model_history_log.csv", append=True)

In [28]:
history = model.fit(train_gen, validation_data=validation_gen, epochs=20, verbose=1,
                    callbacks=[early_stop, rate_reduction, csv_logger])

Epoch 1/20
158/158 [==============================] - 116s 734ms/step - loss: 2.6657 - accuracy: 0.2010 - val_loss: 1.6052 - val_accuracy: 0.4175
Epoch 2/20
158/158 [==============================] - 113s 717ms/step - loss: 1.5995 - accuracy: 0.3718 - val_loss: 1.2269 - val_accuracy: 0.5333
Epoch 3/20
158/158 [==============================] - 113s 715ms/step - loss: 1.3701 - accuracy: 0.4534 - val_loss: 1.1685 - val_accuracy: 0.5651
Epoch 4/20
158/158 [==============================] - 113s 716ms/step - loss: 1.1326 - accuracy: 0.5633 - val_loss: 0.7678 - val_accuracy: 0.7675
Epoch 5/20
158/158 [==============================] - 113s 716ms/step - loss: 0.9830 - accuracy: 0.6256 - val_loss: 0.5727 - val_accuracy: 0.8317
Epoch 6/20
158/158 [==============================] - 113s 717ms/step - loss: 0.8537 - accuracy: 0.6782 - val_loss: 0.6557 - val_accuracy: 0.7698
Epoch 7/20
158/158 [==============================] - 113s 716ms/step - loss: 0.8171 - accuracy: 0.6887 - val_loss: 0.5076 -

In [29]:
import numpy as np

test_gen = img_gen.flow_from_dataframe(test_data,
                                       x_col="image",
                                       y_col="label",
                                       target_size=img_size,
                                       class_mode="categorical",
                                       batch_size=32,
                                       shuffle=False)

prediction = np.argmax(model.predict(test_gen), axis=1)
train_labels = train_gen.class_indices
train_labels = dict((v, k) for k, v in train_labels.items())
predictions = [train_labels[k] for k in prediction]

Found 2700 validated image filenames belonging to 9 classes.


In [30]:
from sklearn.metrics import classification_report
y_test = test_data.label
print(classification_report(y_test, predictions))

model.save("fishes_model.h5", overwrite=True)
print("Saved successfully")

                    precision    recall  f1-score   support

   Black Sea Sprat       0.93      0.85      0.89       300
   Gilt-Head Bream       0.86      0.82      0.84       326
   Hourse Mackerel       0.96      0.98      0.97       287
        Red Mullet       0.98      1.00      0.99       288
     Red Sea Bream       0.86      0.92      0.89       296
          Sea Bass       0.95      0.70      0.80       298
            Shrimp       0.89      1.00      0.94       314
Striped Red Mullet       0.77      0.85      0.81       297
             Trout       0.92      0.98      0.95       294

          accuracy                           0.90      2700
         macro avg       0.90      0.90      0.90      2700
      weighted avg       0.90      0.90      0.90      2700

Saved successfully


In [31]:
from tensorflowjs import converters
converters.save_keras_model(model, "tf-js-model")
print("Saved successfully")

Saved successfully
